In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1579219203575_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-4>

In [51]:
#from pyspark import SparkContext
from pyspark.sql import SparkSession
ss = SparkSession.builder.getOrCreate()


#sc = SparkContext.getOrCreate()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
"""sc.install_pypi_package('h2o-pysparkling-2.4')
sc.install_pypi_package('h2o')
from pysparkling import *
import h2o
ss = SparkSession.builder.getOrCreate()
hc = H2OContext.getOrCreate(ss)"""

In [ ]:
## read from S3

In [52]:
# inner joined data
data1 = ss.read.parquet("s3a://dc-group2/join_final")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
data1 = data1.orderBy(['State_Name', 'County_Name',
                       'State_Code', 'County_Code', 'Date_Local'])
#data1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+-----------+----------+----+--------------------+-----------------+------------------+------------------+-----------------+
|State_Name|County_Name|State_Code|County_Code|Date_Local| AQI|                temp|         humidity|          pressure|        wind_speed|   wind_direction|
+----------+-----------+----------+-----------+----------+----+--------------------+-----------------+------------------+------------------+-----------------+
|   Alabama|  Jefferson|         1|         73|2011-01-01|26.0|  13.100000381469727|78.83333587646484| 991.8375244140625|              3.25|213.0833282470703|
|   Alabama|  Jefferson|         1|         73|2011-01-02|30.0|   2.987499952316284|53.66666793823242| 998.9625244140625|2.0333330631256104|238.3333282470703|
|   Alabama|  Jefferson|         1|         73|2011-01-03|68.0|  2.4916670322418213|50.20833206176758|  999.120849609375|0.9916669726371765|136.2916717529297|
|   Alabama|  Jefferson|         1|         73

In [56]:
data1 = data1.withColumn("State_Code_num", data1['State_Code'].astype('integer'))
data1 = data1.withColumn("County_Code_num", data1['County_Code'].astype('integer'))
data1.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('State_Name', 'string'), ('County_Name', 'string'), ('State_Code', 'string'), ('County_Code', 'string'), ('Date_Local', 'string'), ('AQI', 'double'), ('temp', 'double'), ('humidity', 'double'), ('pressure', 'double'), ('wind_speed', 'double'), ('wind_direction', 'double'), ('State_Code_num', 'int'), ('County_Code_num', 'int')]

In [57]:
data1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

599673

## Feature Engineering

In [58]:
import datetime
from pyspark.sql.functions import year, month, dayofmonth,dayofweek

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
data1_date = data1.withColumn("Date", data1['Date_Local'].astype('Date'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
data1_date.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+-----------+----------+----+--------------------+-----------------+------------------+------------------+-----------------+--------------+---------------+----------+
|State_Name|County_Name|State_Code|County_Code|Date_Local| AQI|                temp|         humidity|          pressure|        wind_speed|   wind_direction|State_Code_num|County_Code_num|      Date|
+----------+-----------+----------+-----------+----------+----+--------------------+-----------------+------------------+------------------+-----------------+--------------+---------------+----------+
|   Alabama|  Jefferson|         1|         73|2011-01-01|26.0|  13.100000381469727|78.83333587646484| 991.8375244140625|              3.25|213.0833282470703|             1|             73|2011-01-01|
|   Alabama|  Jefferson|         1|         73|2011-01-02|30.0|   2.987499952316284|53.66666793823242| 998.9625244140625|2.0333330631256104|238.3333282470703|             1|             73|2011-01

In [61]:
data1_date.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['State_Name', 'County_Name', 'State_Code', 'County_Code', 'Date_Local', 'AQI', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_direction', 'State_Code_num', 'County_Code_num', 'Date']

In [62]:
data1_date = data1_date.select('State_Name',
                               'County_Name',
                               'State_Code_num',
                               'County_Code_num',
                               'Date',
                                year("Date").alias('year'),
                               month("Date").alias('month'),
                               dayofmonth("Date").alias('day'),
                               dayofweek('Date').alias('dow'),
                               'temp',
                               'humidity',
                               'pressure',
                               'wind_speed',
                               'wind_direction',
                               'AQI'
                               
                              )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
data1_date = data1_date.withColumnRenamed('AQI','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
data1_date.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['State_Name', 'County_Name', 'State_Code_num', 'County_Code_num', 'Date', 'year', 'month', 'day', 'dow', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_direction', 'label']

In [65]:
data1_select = data1_date.select(
                               'State_Code_num',
                               'County_Code_num',
                                year("Date").alias('year'),
                                month("Date").alias('month'),
                                dayofmonth("Date").alias('day'),
                                dayofweek('Date').alias('dow'),
                               'temp',
                               'humidity',
                               'pressure',
                               'wind_speed',
                               'wind_direction',
                               'label'
                              )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
# Merging the data with Vector Assembler.
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(outputCol="features", inputCols=data1_select.columns[0:-1]) #except the last col.
data1_va = va.transform(data1_select).select("features", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Train RF model

In [67]:
# train-test split
data1sets = data1_va.randomSplit([0.8, 0.2])
data1_train = data1sets[0].cache()
data1_test = data1sets[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
# Train the model.ntrees=50
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()
rfmodel = rf.fit(data1_train)
print(rfmodel._call_java('toDebugString'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RandomForestRegressionModel (uid=RandomForestRegressor_34d2293154ce) with 20 trees
  Tree 0 (weight 1.0):
    If (feature 1 <= 2.0)
     If (feature 8 <= 885.1333312988281)
      If (feature 8 <= 877.1458435058594)
       If (feature 6 <= 52.37202344621931)
        If (feature 4 <= 1.5)
         Predict: 67.48387096774194
        Else (feature 4 > 1.5)
         Predict: 49.39601769911504
       Else (feature 6 > 52.37202344621931)
        If (feature 7 <= 74.29374694824219)
         Predict: 61.515337423312886
        Else (feature 7 > 74.29374694824219)
         Predict: 147.41256830601094
      Else (feature 8 > 877.1458435058594)
       If (feature 2 <= 2007.5)
        If (feature 10 <= 64.10416507720947)
         Predict: 33.861702127659576
        Else (feature 10 > 64.10416507720947)
         Predict: 155.0066225165563
       Else (feature 2 > 2007.5)
        If (feature 2 <= 2010.5)
         Predict: 171.7578027465668
        Else (feature 2 > 2010.5)
         Predict: 140.92594

In [69]:
rfpred = rfmodel.transform(data1_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 25.8653

In [72]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2") # supports "rmse" (default), "mse", "r2", and "mae"
r2 = evaluator.evaluate(rfpred)
print("R2 on test data = %g" % r2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R2 on test data = 0.29129

In [73]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpred)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Mean Absolute Error (MAE) on test data = 16.7333

In [79]:
# ntrees=100
rf = RandomForestRegressor(numTrees=100)
rfmodel = rf.fit(data1_train)
rfpred = rfmodel.transform(data1_test)
#print(rfmodel._call_java('toDebugString'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpred)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 25.8752
Mean Absolute Error (MAE) on test data = 16.7021

In [81]:
# ntrees=10
rf = RandomForestRegressor(numTrees=10)
rfmodel = rf.fit(data1_train)
rfpred = rfmodel.transform(data1_test)
#print(rfmodel._call_java('toDebugString'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpred)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 25.9239
Mean Absolute Error (MAE) on test data = 16.7486

## add nearest AQI as a feature

In [83]:
## inner data+nearest AQI as a feature
from pyspark.sql.window import Window
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
data1_date.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('State_Name', 'string'), ('County_Name', 'string'), ('State_Code_num', 'int'), ('County_Code_num', 'int'), ('Date', 'date'), ('year', 'int'), ('month', 'int'), ('day', 'int'), ('dow', 'int'), ('temp', 'double'), ('humidity', 'double'), ('pressure', 'double'), ('wind_speed', 'double'), ('wind_direction', 'double'), ('label', 'double')]

In [85]:
data1_lag = data1_date.select(
                               'State_Code_num',
                               'County_Code_num',
                                year("Date").alias('year'),
                                month("Date").alias('month'),
                                dayofmonth("Date").alias('day'),
                                dayofweek('Date').alias('dow'),
                               'temp',
                               'humidity',
                               'pressure',
                               'wind_speed',
                               'wind_direction',
                                lag("label", 1).over(Window.partitionBy(
                                    'State_Code_num','County_Code_num').orderBy('Date')).alias("prev_AQI"),
                               'label'
                              )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
data1_lag.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['State_Code_num', 'County_Code_num', 'year', 'month', 'day', 'dow', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_direction', 'prev_AQI', 'label']

In [87]:
data1_lag = data1_lag.where("prev_AQI is not null")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
va = VectorAssembler(outputCol="features", inputCols=data1_lag.columns[0:-1]) #except the last col.
data1_lag_va = va.transform(data1_lag).select("features", "label")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
# train-test split
data1_lag_sets = data1_lag_va.randomSplit([0.8, 0.2])
data1_lag_train = data1_lag_sets[0].cache()
data1_lag_test = data1_lag_sets[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
# Train the model.ntree=50

rf = RandomForestRegressor()
rfmodel = rf.fit(data1_lag_train)
#print(rfmodel._call_java('toDebugString'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
rfpred = rfmodel.transform(data1_lag_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 43.0379

In [93]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpred)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Mean Absolute Error (MAE) on test data = 13.3504

In [94]:
# Train the model.ntree=100

rf = RandomForestRegressor(numTrees=100)
rfmodel = rf.fit(data1_lag_train)
#print(rfmodel._call_java('toDebugString'))
rfpred = rfmodel.transform(data1_lag_test)
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpred)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 43.0955
Mean Absolute Error (MAE) on test data = 13.4051

In [95]:
# Train the model.ntree=10

rf = RandomForestRegressor(numTrees=10)
rfmodel = rf.fit(data1_lag_train)
#print(rfmodel._call_java('toDebugString'))
rfpred = rfmodel.transform(data1_lag_test)
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpred)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 42.8275
Mean Absolute Error (MAE) on test data = 13.5292

In [96]:
# Train the model.ntree=200

rf = RandomForestRegressor(numTrees=200)
rfmodel = rf.fit(data1_lag_train)
#print(rfmodel._call_java('toDebugString'))
rfpred = rfmodel.transform(data1_lag_test)
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpred)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 43.1106
Mean Absolute Error (MAE) on test data = 13.4208

## add prev_3 AQI as features

In [97]:
data1_lag3 = data1_date.select(
                               'State_Code_num',
                               'County_Code_num',
                                year("Date").alias('year'),
                                month("Date").alias('month'),
                                dayofmonth("Date").alias('day'),
                                dayofweek('Date').alias('dow'),
                               'temp',
                               'humidity',
                               'pressure',
                               'wind_speed',
                               'wind_direction',
                                lag("label", 1).over(Window.partitionBy(
                                    'State_Code_num','County_Code_num').orderBy('Date')).alias("prev1_AQI"),
    lag("label", 2).over(Window.partitionBy(
                                    'State_Code_num','County_Code_num').orderBy('Date')).alias("prev2_AQI"),
    lag("label", 3).over(Window.partitionBy(
                                    'State_Code_num','County_Code_num').orderBy('Date')).alias("prev3_AQI"),
                               'label'
                              )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [102]:
data1_lag3 = data1_lag3.where("prev1_AQI is not null and prev2_AQI is not null and prev3_AQI is not null")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
va = VectorAssembler(outputCol="features", inputCols=data1_lag3.columns[0:-1]) #except the last col.
data1_lag3_va = va.transform(data1_lag3).select("features", "label")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
# train-test split
data1_lag3_sets = data1_lag3_va.randomSplit([0.8, 0.2])
data1_lag3_train = data1_lag3_sets[0].cache()
data1_lag3_test = data1_lag3_sets[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
# Train the model.ntree=100

rf = RandomForestRegressor(numTrees=100)
rfmodel = rf.fit(data1_lag3_train)
#print(rfmodel._call_java('toDebugString'))
rfpred = rfmodel.transform(data1_lag3_test)
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpred)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 21.2591
Mean Absolute Error (MAE) on test data = 12.9686

In [2]:
sc.install_pypi_package('h2o-pysparkling-2.4')
sc.install_pypi_package('h2o')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing /var/lib/livy/.cache/pip/wheels/0d/2f/d2/7e0c36c53b57ca597430858ab3d72e1a612a10780016ff6765/h2o_pysparkling_2.4-3.28.0.1.post1-py2.py3-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e/tabulate-0.8.6-cp36-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471/pyspark-2.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c9/dc/45cdef1b4d119eb96316b3117e6d5708a08029992b2fee2c143c7a0a5cc5/colorama-0.4.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e/future-0.18.2-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/53/c737818eb9a7dc32a7cd4f1396e787bd94200c3997c72c1dbe0285

In [3]:
from pysparkling import *
import h2o
h2o.init()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Checking whether there is an H2O instance running at http://localhost:54321 . connected.
--------------------------  ------------------------------------------------------------------
H2O cluster uptime:         1 hour 7 mins
H2O cluster timezone:       Etc/UCT
H2O data parsing timezone:  UTC
H2O cluster version:        3.28.0.1
H2O cluster version age:    1 month
H2O cluster name:           H2O_from_python_livy_1hnnw4
H2O cluster total nodes:    1
H2O cluster free memory:    5.800 Gb
H2O cluster total cores:    8
H2O cluster allowed cores:  8
H2O cluster status:         locked, healthy
H2O connection url:         http://localhost:54321
H2O connection proxy:       {'http': None, 'https': None}
H2O internal security:      False
H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, TargetEncoder, Core V4
Python version:             3.6.8 final
--------------------------  ------------------------------------------------------------------

In [ ]:
data1_lag_rdd = data1_lag.rdd

In [ ]:
data1_lag_rdd.saveAsTextFile("s3a://dc-group2/data1_lag_rdd")

In [ ]:
h2o.init()

In [ ]:
data1_lag_h2o = h2o.import_file("s3a://dc-group2/data1_lag_rdd")

In [ ]:
data1_lag_h2o.summary()

In [ ]:
data1_lag.write.csv("s3a://dc-group2/data1_lag.csv")

In [108]:
data1_lag3.write.csv("s3a://dc-group2/data1_lag3.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
data1_lag_h2o = h2o.import_file("s3a://dc-group2/data1_lag.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Parse progress: [#########################################################] 100%

In [5]:
data1_lag_h2o.set_names(['State_Code_num', 'County_Code_num', 'year', 'month',
                         'day', 'dow', 'temp', 'humidity', 'pressure', 'wind_speed',
                         'wind_direction', 'prev_AQI', 'label'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  State_Code_num    County_Code_num    year    month    day    dow     temp    humidity    pressure    wind_speed    wind_direction    prev_AQI    label
----------------  -----------------  ------  -------  -----  -----  -------  ----------  ----------  ------------  ----------------  ----------  -------
              36                 83    2002        3     15      6  49.0417     65.25       957           5.34167          169.208           35       37
              36                 83    2002        3     16      7  36.7083     84.8333     960.125       3.79583          163               37       29
              36                 83    2002        3     17      1  28.5417     60.25       973.083       1.97917           62.8333          29       35
              36                 83    2002        3     18      2  30.5        76.125      968           3.175            163.458           35       34
              36                 83    2002        3     19      3  33.125      77

In [109]:
data1_lag3_h2o = h2o.import_file("s3a://dc-group2/data1_lag3.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Parse progress: [#########################################################] 100%

In [111]:
data1_lag3_h2o.set_names(['State_Code_num', 'County_Code_num', 'year', 'month', 'day',
                         'dow', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_direction',
                         'prev1_AQI', 'prev2_AQI', 'prev3_AQI', 'label'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  State_Code_num    County_Code_num    year    month    day    dow     temp    humidity    pressure    wind_speed    wind_direction    prev1_AQI    prev2_AQI    prev3_AQI    label
----------------  -----------------  ------  -------  -----  -----  -------  ----------  ----------  ------------  ----------------  -----------  -----------  -----------  -------
              36                 83    2002        3     17      1  28.5417     60.25       973.083       1.97917           62.8333           29           37           35       35
              36                 83    2002        3     18      2  30.5        76.125      968           3.175            163.458            35           29           37       34
              36                 83    2002        3     19      3  33.125      77.4167     969.25        1.69167          143.708            34           35           29       31
              36                 83    2002        3     20      4  31          86.9167     961.875 

In [6]:
data1_lag_train, data1_lag_test = data1_lag_h2o.split_frame(ratios = [.8], seed = 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
predictors = data1_lag_train.names[:]
response = "label"
predictors.remove(response)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
predictors

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['State_Code_num', 'County_Code_num', 'year', 'month', 'day', 'dow', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_direction', 'prev_AQI']

In [9]:
data1_lag_train.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  State_Code_num    County_Code_num    year    month    day    dow     temp    humidity    pressure    wind_speed    wind_direction    prev_AQI    label
----------------  -----------------  ------  -------  -----  -----  -------  ----------  ----------  ------------  ----------------  ----------  -------
              36                 83    2002        3     15      6  49.0417     65.25       957           5.34167           169.208          35       37
              36                 83    2002        3     16      7  36.7083     84.8333     960.125       3.79583           163              37       29
              36                 83    2002        3     18      2  30.5        76.125      968           3.175             163.458          35       34
              36                 83    2002        3     19      3  33.125      77.4167     969.25        1.69167           143.708          34       31
              36                 83    2002        3     20      4  31          86

In [12]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
model_gbm = H2OGradientBoostingEstimator(ntrees=50,
                                         max_depth=6,
                                         learn_rate=0.1,
                                         )

model_gbm.train(x=predictors,
                y=response,
                training_frame=data1_lag_train,
                validation_frame=data1_lag_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gbm Model Build progress: [###############################################] 100%

In [13]:
model_gbm

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1579224764540_2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          26039                  6            6            6             13            59            36.82


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 575.1760046658201
RMSE: 23.982827286744573
MAE: 12.049059788703016
RMSLE: NaN
Mean Residual Deviance: 575.1760046658201

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 732.4899316558377
RMSE: 27.06455119997074
MAE: 12.242259437538323
RMSLE: NaN
Mean Residual Deviance: 732.4899316558377

Scoring History: 
    timestamp

In [14]:

model_gbm = H2OGradientBoostingEstimator(ntrees=100,
                                         max_depth=6,
                                         learn_rate=0.1,
                                         )

model_gbm.train(x=predictors,
                y=response,
                training_frame=data1_lag_train,
                validation_frame=data1_lag_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gbm Model Build progress: [###############################################] 100%

In [15]:
model_gbm

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1579224764540_3


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                100                         40724                  6            6            6             10            59            27.79


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 496.23672227154236
RMSE: 22.276371389244307
MAE: 11.805238136502648
RMSLE: NaN
Mean Residual Deviance: 496.23672227154236

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 728.6460232725884
RMSE: 26.993444079490644
MAE: 12.016053149856196
RMSLE: NaN
Mean Residual Deviance: 728.6460232725884

Scoring History: 
    timest

In [16]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
model_dl = H2ODeepLearningEstimator(variable_importances=True,
                                    loss="Automatic")

model_dl.train(x=predictors,
               y=response,
               training_frame=data1_lag_train,
               validation_frame=data1_lag_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

deeplearning Model Build progress: [######################################] 100%

In [17]:
model_dl

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1579224764540_4


Status of Neuron Layers: predicting label, regression, gaussian distribution, Quadratic loss, 43,001 weights/biases, 516.3 KB, 4,802,613 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  -------------------  -----------------------
    1        12       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.01185059714844101    0.02420925348997116    0.0         0.014853620929082656   0.6004297733306885   0.23964601905099914  0.433846116065979
    3        200      Rectifier  0.0        0.0   0.0   0.05970013060117553    0.089534044

In [48]:
from h2o.estimators.xgboost import H2OXGBoostEstimator
model_xg = H2OXGBoostEstimator()
model_xg.train(x=predictors,
               y="label",
               training_frame=data1_lag_train,
               validation_frame=data1_lag_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

xgboost Model Build progress: [###########################################] 100%

In [49]:
model_xg

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1579233806610_1


Model Summary: 
    number_of_trees
--  -----------------
    50


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 314.9533438845186
RMSE: 17.74692491347497
MAE: 11.067762413489184
RMSLE: NaN
Mean Residual Deviance: 314.9533438845186

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 1697.03962067154
RMSE: 41.195140741009006
MAE: 11.492590799365352
RMSLE: NaN
Mean Residual Deviance: 1697.03962067154

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance    validation_rmse    validation_mae    validation_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------  -----------------  ----------------  ---------------------
    2020-01-17 04:37:58  0.001 sec   0                  61.0948          49.1558         3732.57       

In [117]:
## performance on LA

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
data1_lag_test_la = data1_lag_test[(data1_lag_test['State_Code_num']==6)&(data1_lag_test['County_Code_num']==37),:]
data1_lag_test_la

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  State_Code_num    County_Code_num    year    month    day    dow     temp    humidity    pressure    wind_speed    wind_direction    prev_AQI    label
----------------  -----------------  ------  -------  -----  -----  -------  ----------  ----------  ------------  ----------------  ----------  -------
               6                 37    2007        1      4      5  53.5417     78.3958     480.471       3.17853           172.093          67       65
               6                 37    2007        1      5      6  51.5469     22.7708     483.673       6.64931           203.513          65       44
               6                 37    2007        1      7      1  57.0729     33.3889     487.979       3.77821           190.308          82       66
               6                 37    2007        1     11      5  51.6354     69.6806     477.55        3.725             151.074         102       52
               6                 37    2007        1     12      6  44.7292     30

In [31]:
model_dl_la = H2ODeepLearningEstimator(variable_importances=True,
                                    loss="Automatic")

model_dl_la.train(x=predictors,
               y=response,
               training_frame=data1_lag_train,
               validation_frame=data1_lag_test_la)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

deeplearning Model Build progress: [######################################] 100%

In [32]:
model_dl_la

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1579224764540_5


Status of Neuron Layers: predicting label, regression, gaussian distribution, Quadratic loss, 43,001 weights/biases, 516.3 KB, 4,200,904 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight            weight_rms           mean_bias           bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  --------------------  ----------  ---------------------  -------------------  ------------------  -----------------------
    1        12       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.018285708628476036   0.04162207245826721   0.0         0.0016365854969232411  0.40053272247314453  0.2390218242321387  0.26254284381866455
    3        200      Rectifier  0.0        0.0   0.0   0.06667813490682076    0.0987009108066

## H2O with data1_lag3_h2o

In [112]:
data1_lag3_train, data1_lag3_test = data1_lag3_h2o.split_frame(ratios = [.8], seed = 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
predictors = data1_lag3_train.names[:]
response = "label"
predictors.remove(response)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
model_dl_lag3 = H2ODeepLearningEstimator(variable_importances=True,
                                    loss="Automatic")

model_dl_lag3.train(x=predictors,
               y=response,
               training_frame=data1_lag3_train,
               validation_frame=data1_lag3_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

deeplearning Model Build progress: [######################################] 100%

In [116]:
model_dl_lag3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1579233806610_2


Status of Neuron Layers: predicting label, regression, gaussian distribution, Quadratic loss, 43,401 weights/biases, 521.7 KB, 4,797,840 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  -------------------  -----------------------
    1        14       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.018285798708613063   0.04999564588069916    0.0         0.027204224719124405  0.5403749942779541   0.18337972208140235  0.4484602212905884
    3        200      Rectifier  0.0        0.0   0.0   0.04734868463571366    0.08254998922

In [119]:
## performance on LA
data1_lag3_test_la = data1_lag3_test[(data1_lag3_test['State_Code_num']==6)&(data1_lag3_test['County_Code_num']==37),:]
data1_lag3_test_la

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  State_Code_num    County_Code_num    year    month    day    dow     temp    humidity    pressure    wind_speed    wind_direction    prev1_AQI    prev2_AQI    prev3_AQI    label
----------------  -----------------  ------  -------  -----  -----  -------  ----------  ----------  ------------  ----------------  -----------  -----------  -----------  -------
               6                 37    2007        1      5      6  51.5469     22.7708     483.673       6.64931           203.513           65           67           81       44
               6                 37    2007        1      6      7  50.724      32.0764     486.444       3.23009           166.474           44           65           67       82
               6                 37    2007        1     10      4  56.2813     52.9306     479.548       2.45011           155.993           87           70           66      102
               6                 37    2007        1     11      5  51.6354     69.6806     477.55  

In [121]:
model_dl_lag3_la = H2ODeepLearningEstimator(variable_importances=True,
                                    loss="Automatic")

model_dl_lag3_la.train(x=predictors,
               y=response,
               training_frame=data1_lag3_train,
               validation_frame=data1_lag3_test_la)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

deeplearning Model Build progress: [######################################] 100%

In [122]:
model_dl_lag3_la

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1579233806610_3


Status of Neuron Layers: predicting label, regression, gaussian distribution, Quadratic loss, 43,401 weights/biases, 521.7 KB, 3,099,437 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms          mean_bias           bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  ------------------  ------------------  -----------------------
    1        14       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.01128955671892202    0.026477627456188202   0.0         0.011851654942431813  0.338104248046875   0.3200264209836081  0.2297118902206421
    3        200      Rectifier  0.0        0.0   0.0   0.05180659398104541    0.08135521411895752

In [123]:
model_dl_lag3.varimp(use_pandas=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('pressure', 1.0, 1.0, 0.24352833540789529), ('wind_speed', 0.49859076738357544, 0.49859076738357544, 0.12142097963066727), ('County_Code_num', 0.3483409583568573, 0.3483409583568573, 0.08483089374303643), ('temp', 0.31487762928009033, 0.31487762928009033, 0.07668162491576475), ('wind_direction', 0.2978503108024597, 0.2978503108024597, 0.07253499039044727), ('State_Code_num', 0.2913719117641449, 0.2913719117641449, 0.07095731665653836), ('humidity', 0.26199865341186523, 0.26199865341186523, 0.06380409594450162), ('year', 0.24647609889507294, 0.24647609889507294, 0.06002391408174889), ('month', 0.24184292554855347, 0.24184292554855347, 0.05889560508901478), ('prev1_AQI', 0.19858261942863464, 0.19858261942863464, 0.048360494750394965), ('prev2_AQI', 0.12196200340986252, 0.12196200340986252, 0.029701203673415867), ('prev3_AQI', 0.11005360633134842, 0.11005360633134842, 0.026801171555509085), ('dow', 0.09128127247095108, 0.09128127247095108, 0.022229576338765253), ('day', 0.08306958526372

## AUTOML:
- too time consuming

In [43]:
data1_lag_h2o = h2o.import_file("s3a://dc-group2/data1_lag.csv")

data1_lag_h2o.set_names(['State_Code_num', 'County_Code_num', 'year', 'month',
                         'day', 'dow', 'temp', 'humidity', 'pressure', 'wind_speed',
                         'wind_direction', 'prev_AQI', 'label'])

data1_lag_train, data1_lag_test = data1_lag_h2o.split_frame(ratios = [.8], seed = 1)

predictors = data1_lag_train.names[:]
response = "label"
predictors.remove(response)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Parse progress: [#########################################################] 100%

In [44]:
from h2o.automl import H2OAutoML
model_automl = H2OAutoML(max_runtime_secs = 900, seed = 1, nfolds = 3) # seed : Set a seed for reproducibility
model_automl.train(x=predictors,
                y=response,
                training_frame=data1_lag_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AutoML progress: [########################################################] 100%

In [45]:
model_automl.leaderboard

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model_id                            mean_residual_deviance     rmse      mse      mae    rmsle
--------------------------------  ------------------------  -------  -------  -------  -------
XGBoost_1_AutoML_20200117_040500                   735.284  27.1161  735.284  10.9672      nan

[1 row x 6 columns]

In [46]:

model_automl2 = H2OAutoML(max_runtime_secs = 900, seed = 1) # seed : Set a seed for reproducibility
model_automl2.train(x=predictors,
                y=response,
                training_frame=data1_lag_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AutoML progress: [########################################################] 100%

In [47]:
model_automl2.leaderboard

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model_id                            mean_residual_deviance     rmse      mse      mae    rmsle
--------------------------------  ------------------------  -------  -------  -------  -------
XGBoost_1_AutoML_20200117_042119                   716.675  26.7708  716.675  10.9654      nan

[1 row x 6 columns]